## WikiDataより，Bird（ID：Q5117）を直接的または間接的に親に持つ記事のURLを取得

##### ・data_external/query.tsvがURL一覧

In [7]:
with open("data_external/query.tsv") as f:
    url_list = f.readlines()[1:]
with open('sh1_makejson.sh', 'w', encoding='UTF-8') as f:
    f.write("#!/bin/bash\n")
    
    for url in url_list:
        ID = url.replace("http://www.wikidata.org/entity/","")
        command = "curl -sLH 'Accept: application/json' "+url+" | jq . > data_json/"+ID+".json"
        f.write(command+"\n")

## sh1_makejson.shを実行

##### ・コマンドラインにて
##### ・これが回り終えるまでに3日かかる

## 実行結果を順に複数のdata_extract/data_json_n.pickle(ただしn = 1,2,...,7,8)ファイルに保存

##### ・data_json/以下のデータがある程度たまったらdata_json_n/に移動する

In [18]:
import os
import json
import pickle

def logmaker(file_dir,file,error):
    with open(file_dir+"/"+file+"_log.txt", 'w',encoding="utf-8") as log_f:
        log_f.write(error+"\n")

def Make_jsonpikle(file_dir):
    files = os.listdir(file_dir)
    files_1 = [file.replace(".json","") for file in files]
    all_bird_dict = []

    for i in range(len(files_1)):
        file = files_1[i]
        print('\r%d / %d' %(i, len(files_1)), end='')
        try:
            with open(file_dir+'/'+file+".json", 'r',encoding="utf-8") as rf:
                json_dict = json.load(rf)
        except:
            logmaker(file_dir,file,"read json")
            continue


        bird_dict = {}
        url = "https://www.wikidata.org/wiki/"+file
        bird_dict["id"] = file
        try:
            bird_dict["en_name"] = json_dict['entities'][file]["labels"]["en"]["value"]
        except:
            bird_dict["en_name"] = None
            logmaker(file_dir,file,"en_name")
        try:
            bird_dict["ja_name"] = json_dict['entities'][file]["labels"]["ja"]["value"]
        except:
            bird_dict["ja_name"] = None

        bird_dict["en_aliases"] = {}
        cnt1 = 0
        try:
            for d in json_dict['entities'][file]["aliases"]["en"]:
                bird_dict["en_aliases"][cnt1] = d["value"]
                cnt1 = cnt1 + 1
        except:
            pass
        bird_dict["ja_aliases"] = {}
        cnt2 = 0
        try:
            for d in json_dict['entities'][file]["aliases"]["ja"]:
                bird_dict["ja_aliases"][cnt2]=d["value"]
                cnt2 = cnt2 + 1
        except:
            pass


        bird_dict["img_urls"] = {}
        cnt3 = 0
        try:
            for img in json_dict['entities'][file]["claims"]["P18"]:
                img_url = url+"#/media/File:"+img["mainsnak"]["datavalue"]["value"].replace(" ","%20")
                bird_dict["img_urls"][cnt3]=img_url
                cnt3 = cnt3 + 1
        except:
            pass

        try:
            bird_dict["taxon_rank"] = json_dict['entities'][file]["claims"]["P105"][0]["mainsnak"]["datavalue"]["value"]["id"]
        except:
            bird_dict["taxon_rank"] = None
            logmaker(file_dir,file,"taxon_rank")

        try:
            bird_dict["taxon_name"] = json_dict['entities'][file]["claims"]["P225"][0]["mainsnak"]["datavalue"]["value"]
        except:
            bird_dict["taxon_name"] = None
            logmaker(file_dir,file,"taxon_name")

        try:
            bird_dict["parent_taxon"] = json_dict['entities'][file]["claims"]["P171"][0]["mainsnak"]["datavalue"]["value"]["id"]
        except:
            bird_dict["parent_taxon"] = None
            logmaker(file_dir,file,"parent_taxon")
        all_bird_dict.append(bird_dict)


    with open("data_extract/data_json/"+file_dir+".pickle", 'wb') as bf:
        pickle.dump(all_bird_dict,bf)

In [21]:
for i in [1,2,3,4,5,6,7,8]:
    Make_jsonpikle("data_json/data_json_"+str(i))

##### ・この時点でdata_json_n(n=1)の各ディレクトリは不要なのでtrushへ

# 全てのdata_json_n.pickleを結合し，data_extract/ontrogy.pickleを作成

In [ ]:
import pickle

data = []
for i in [1,2,3,4,5,6,7,8]:
    with open("data_extract/data_json_"+i+".pickle","rb") as f:
        data_n = pickle.load(f)
        data = data+data_n
        
data_d = {}
for d in data:
    data_d[d["id"]]=d

ID_d = dict()
nodata_set = set()
for k in ID_set:
    if k in data_d:
        ID_d[k] = data_d[k]["en_name"]
    else:
        nodata_set.add(k)

ID_d_ja = dict()
nodata_set_ja = set()
for k in ID_set:
    if k in data_d:
        ID_d_ja[k] = data_d[k]["ja_name"]
    else:
        nodata_set_ja.add(k)

In [ ]:
non_data = []
for d in data:
    try:
        d["taxon_rank_name"] = ID_d[d["taxon_rank"]]
        d["taxon_rank_ja_name"] = ID_d_ja[d["taxon_rank"]]

        d["parent_taxon_name"] = ID_d[d["parent_taxon"]]
        d["parent_taxon_ja_name"] = ID_d_ja[d["parent_taxon"]]


        s = ""
        for k,v in d["en_aliases"].items():
            if s == "":
                s = s + v
            else:
                s = s + "," +v
        d["en_aliases"] = s

        s = ""
        for k,v in d["ja_aliases"].items():
            if s == "":
                s = s + v
            else:
                s = s + "," +v
        d["ja_aliases"] = s

        s = ""
        for k,v in d["img_urls"].items():
            if s == "":
                s = s + v
            else:
                s = s + " " +v
        d["img_urls"] = s
    except:
        non_data.append(d["taxon_rank"])

# ontrogy.pickleにおけるparent_taxon一覧を作成

##### ・(1)既存のpickleを取得

##### ・(2)all_p_taxonのうち，既に記事取得済みのidに存在しないものをabsent_p_taxonに格納

In [ ]:
#辞書データ
import pickle
# with open("data_extract/ontology.pickle", 'rb') as f:
# with open("data_extract/ontology_v2.pickle", 'rb') as f:
# with open("data_extract/ontology_v3.pickle", 'rb') as f:
# with open("data_extract/ontology_v4.pickle", 'rb') as f:
# with open("data_extract/ontology_v5.pickle", 'rb') as f:
with open("data_extract/ontology_v6.pickle", 'rb') as f:
    data_origin = pickle.load(f)

all_id = set()
all_p_taxon = set()
for d in data_origin:
    all_id.add(d["id"])
    all_p_taxon.add(d["parent_taxon"])
    
absent_p_taxon = all_p_taxon-all_id
len(absent_p_taxon)

##### ・(3)all_p_taxonに存在しないものを取得するshellファイルを作成し，実行

In [ ]:
# with open('sh3_absent_p_taxon.sh', 'w', encoding='UTF-8') as f:
# with open('sh4_absent_p_taxon.sh', 'w', encoding='UTF-8') as f:
# with open('sh5_absent_p_taxon.sh', 'w', encoding='UTF-8') as f:
# with open('sh6_absent_taxon.sh', 'w', encoding='UTF-8') as f:
    f.write("#!/bin/bash\n")
    
    for item in absent_p_taxon:
        i = str(item)
#         command = "curl -sLH 'Accept: application/json' http://www.wikidata.org/entity/"+i+" | jq . > data_json_10/"+i+".json"
#         command = "curl -sLH 'Accept: application/json' http://www.wikidata.org/entity/"+i+" | jq . > data_json_11/"+i+".json"
#         command = "curl -sLH 'Accept: application/json' http://www.wikidata.org/entity/"+i+" | jq . > data_json_12/"+i+".json"
#         command = "curl -sLH 'Accept: application/json' http://www.wikidata.org/entity/"+i+" | jq . > data_json_13/"+i+".json"
        f.write(command+"\n")

In [ ]:
# !bash sh3_absent_p_taxon.sh
# !bash sh4_absent_p_taxon.sh
# !bash sh5_absent_p_taxon.sh #None.jsonができてしまったので，それは削除
# !bash sh6_absent_p_taxon.sh #None.jsonができてしまったので，それは削除

In [ ]:
# Make_jsonpikle("data_json/data_json_"+str(10))
# Make_jsonpikle("data_json/data_json_"+str(11))
# Make_jsonpikle("data_json/data_json_"+str(12))
# Make_jsonpikle("data_json/data_json_"+str(13))

##### ・上記(1)~(3)の工程を7回目まで，またはall_p_taxonの要素がQ5117のみになるまで継続

In [ ]:
## 

##### ・整形

In [ ]:
data_d = {}
for d in data:
    data_d[d["id"]]=d

ID_list = []
for v in data:
    ID_list.append(v["taxon_rank"])
    ID_list.append(v["parent_taxon"])
ID_set = set(ID_list)

ID_d = dict()
nodata_set = set()
for k in ID_set:
    if k in data_d:
        ID_d[k] = data_d[k]["en_name"]
    else:
        nodata_set.add(k)
        
ID_d_ja = dict()
nodata_set_ja = set()
for k in ID_set:
    if k in data_d:
        ID_d_ja[k] = data_d[k]["ja_name"]
    else:
        nodata_set_ja.add(k)